### Set environment variables before running

In [1]:
%env PGDATABASE=postgres
%env PGHOST=<your_IP>
%env PGPORT=5432
%env PGUSER=postgres
%env PGPASSWORD=<your_password>
%env PGINSTANCE=<your_instance>

env: PGDATABASE=postgres
env: PGHOST=10.44.48.7
env: PGPORT=5432
env: PGUSER=postgres
env: PGPASSWORD=cs327e!
env: PGINSTANCE=pg


### Download Dataset

In [4]:
!gsutil cp gs://cs327e-open-access/postgres_college.zip /home/jupyter/snippets

Copying gs://cs327e-open-access/postgres_college.zip...
/ [1 files][  1.6 KiB/  1.6 KiB]                                                
Operation completed over 1 objects/1.6 KiB.                                      


In [ ]:
!unzip postgres_college.zip

Archive:  postgres_college.zip
replace postgres_college/class.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

### Create tables

In [2]:
!psql -c "\i /home/jupyter/snippets/create_postgres_tables.sql"

psql:/home/jupyter/snippets/create_tables.sql:1: NOTICE:  drop cascades to 4 other objects
DETAIL:  drop cascades to table college.class
drop cascades to table college.instructor
drop cascades to table college.student
drop cascades to table college.teaches
DROP SCHEMA
CREATE SCHEMA
SET
CREATE TABLE
CREATE TABLE
CREATE TABLE
CREATE TABLE


In [3]:
!psql -c "\d college.Class"

                       Table "college.class"
 Column  |          Type          | Collation | Nullable | Default 
---------+------------------------+-----------+----------+---------
 sid     | character varying(10)  |           |          | 
 cno     | character(6)           |           |          | 
 cname   | character varying(200) |           |          | 
 credits | integer                |           |          | 
 grade   | character(2)           |           |          | 



In [4]:
!psql -c "\d college.Student"

                     Table "college.student"
 Column |          Type          | Collation | Nullable | Default 
--------+------------------------+-----------+----------+---------
 sid    | character varying(10)  |           |          | 
 fname  | character varying(200) |           |          | 
 lname  | character varying(200) |           |          | 
 dob    | character varying(20)  |           |          | 
 status | character(3)           |           |          | 



In [5]:
!psql -c "\d college.Instructor"

                     Table "college.instructor"
   Column   |         Type          | Collation | Nullable | Default 
------------+-----------------------+-----------+----------+---------
 tid        | character varying(10) |           |          | 
 instructor | character varying(50) |           |          | 
 dept       | character varying(50) |           |          | 



In [6]:
!psql -c "\d college.Teaches"

                     Table "college.teaches"
 Column |         Type          | Collation | Nullable | Default 
--------+-----------------------+-----------+----------+---------
 tid    | character varying(10) |           |          | 
 cno    | character(6)          |           |          | 



### Populate Database

In [7]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/class.csv --database=postgres --table=college.Class --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/class.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-fa2021/instances/pg].


In [8]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/student.csv --database=postgres --table=college.Student --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/student.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-fa2021/instances/pg].


In [9]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/instructor.csv --database=postgres --table=college.Instructor --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/instructor.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-fa2021/instances/pg].


In [10]:
!gcloud sql import csv $PGINSTANCE gs://cs327e-open-access/postgres_college/teaches.csv --database=postgres --table=college.Teaches --quiet

Importing data into Cloud SQL instance...done.                                 
Imported data from [gs://cs327e-open-access/postgres_college/teaches.csv] into [https://sqladmin.googleapis.com/sql/v1beta4/projects/cs327e-fa2021/instances/pg].


### Verify data load

In [11]:
!psql -c "select count(*) from college.Class"

 count 
-------
    16
(1 row)



In [12]:
!psql -c "select count(*) from college.Student"

 count 
-------
    12
(1 row)



In [13]:
!psql -c "select count(*) from college.Instructor"

 count 
-------
    10
(1 row)



In [14]:
!psql -c "select count(*) from college.Teaches"

 count 
-------
    12
(1 row)



### Exercise: Decompose Class table into new Class and Takes tables (as specified by data model)

In [15]:
!psql -c "select * from college.Class order by cno"

   sid    |  cno   |                 cname                 | credits | grade 
----------+--------+---------------------------------------+---------+-------
 sudeepa4 | CS303E | Elements of Computers and Programming |       3 | 
 jc       | CS313E | Elements of Software Design           |       3 | 
 paulg    | CS313E | Elements of Software Design           |       3 | 
 paulg    | CS326E | Elements of Networking                |       3 | A-
 jerryh   | CS327E | Elements of Databases                 |       3 | B 
 jc       | CS327E | Elements of Databases                 |       3 | 
 paulg    | CS329E | Elements of Web Programming           |       3 | A 
 jerryh   | CS329E | Elements of Web Programming           |       3 | A-
 kev18    | CS329E | Elements of Web Programming           |       3 | 
 jc       | CS330E | Elements of Software Engineering      |       3 | 
 bzen26   | CS330E | Elements of Software Engineering      |       3 | B+
 kev18    | CS347  | Data Management      

In [16]:
!psql -c "create table college.Takes as select sid, cno, grade from college.Class"

SELECT 16


In [ ]:
!psql -c "create table college.Class2 as select distinct cno, cname, credits from college.Class order by cno"

### Drop and rename Class tables

In [17]:
!psql -c "drop table college.Class"

DROP TABLE


In [18]:
!psql -c "alter table college.Class2 rename to Class"

ALTER TABLE


### Add primary keys

In [19]:
!psql -c "alter table college.Class add primary key (cno)"

ALTER TABLE


In [20]:
!psql -c "alter table college.Student add primary key (sid)"

ALTER TABLE


In [21]:
!psql -c "alter table college.Takes add primary key (sid, cno)"

ALTER TABLE


In [22]:
!psql -c "alter table college.Instructor add primary key (tid)"

ALTER TABLE


In [23]:
!psql -c "alter table college.Teaches add primary key (tid, cno)"

ALTER TABLE


### Add foreign keys

In [24]:
!psql -c "alter table college.Takes add foreign key (sid) references college.Student (sid)"

ALTER TABLE


In [25]:
!psql -c "alter table college.Takes add foreign key (cno) references college.Class (cno)"

ALTER TABLE


In [26]:
!psql -c "alter table college.Teaches add foreign key (tid) references college.Instructor (tid)"

ALTER TABLE


In [29]:
!psql -c "alter table college.Teaches add foreign key (cno) references college.Class (cno)"

ALTER TABLE


In [28]:
!psql -c "insert into college.Class (cno, cname, credits) values ('CS331E', 'Elements of Mobile Computing', 3)"

INSERT 0 1


### Describe final tables

In [30]:
!psql -c "\d college.Student"

                     Table "college.student"
 Column |          Type          | Collation | Nullable | Default 
--------+------------------------+-----------+----------+---------
 sid    | character varying(10)  |           | not null | 
 fname  | character varying(200) |           |          | 
 lname  | character varying(200) |           |          | 
 dob    | character varying(20)  |           |          | 
 status | character(3)           |           |          | 
Indexes:
    "student_pkey" PRIMARY KEY, btree (sid)
Referenced by:
    TABLE "college.takes" CONSTRAINT "takes_sid_fkey" FOREIGN KEY (sid) REFERENCES college.student(sid)



In [31]:
!psql -c "\d college.Class"

                       Table "college.class"
 Column  |          Type          | Collation | Nullable | Default 
---------+------------------------+-----------+----------+---------
 cno     | character(6)           |           | not null | 
 cname   | character varying(200) |           |          | 
 credits | integer                |           |          | 
Indexes:
    "class_pkey" PRIMARY KEY, btree (cno)
Referenced by:
    TABLE "college.takes" CONSTRAINT "takes_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    TABLE "college.teaches" CONSTRAINT "teaches_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)



In [32]:
!psql -c "\d college.Takes"

                      Table "college.takes"
 Column |         Type          | Collation | Nullable | Default 
--------+-----------------------+-----------+----------+---------
 sid    | character varying(10) |           | not null | 
 cno    | character(6)          |           | not null | 
 grade  | character(2)          |           |          | 
Indexes:
    "takes_pkey" PRIMARY KEY, btree (sid, cno)
Foreign-key constraints:
    "takes_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    "takes_sid_fkey" FOREIGN KEY (sid) REFERENCES college.student(sid)



In [33]:
!psql -c "\d college.Instructor"

                     Table "college.instructor"
   Column   |         Type          | Collation | Nullable | Default 
------------+-----------------------+-----------+----------+---------
 tid        | character varying(10) |           | not null | 
 instructor | character varying(50) |           |          | 
 dept       | character varying(50) |           |          | 
Indexes:
    "instructor_pkey" PRIMARY KEY, btree (tid)
Referenced by:
    TABLE "college.teaches" CONSTRAINT "teaches_tid_fkey" FOREIGN KEY (tid) REFERENCES college.instructor(tid)



In [34]:
!psql -c "\d college.Teaches"

                     Table "college.teaches"
 Column |         Type          | Collation | Nullable | Default 
--------+-----------------------+-----------+----------+---------
 tid    | character varying(10) |           | not null | 
 cno    | character(6)          |           | not null | 
Indexes:
    "teaches_pkey" PRIMARY KEY, btree (tid, cno)
Foreign-key constraints:
    "teaches_cno_fkey" FOREIGN KEY (cno) REFERENCES college.class(cno)
    "teaches_tid_fkey" FOREIGN KEY (tid) REFERENCES college.instructor(tid)

